In [ ]:
import pandas as pd
import datetime

def clean_mv(files):
    if type(files) == str:
        files = [files]
    dfs = []
    for file_path in files:
        # Initialize variables
        processed_data = []  # Reset the processed data list
        current_date = None  # Reset the current date
        lines = []
        data_atendimento = anomes = hora = atendimento_codigo = paciente_codigo = paciente_nome = convenio = medico_codigo = medico_nome = centro_de_custos = ''

        # Open the file and process each line
        with open(f'{file_path}', 'r', encoding='ISO-8859-1') as file:
            # headers = next(file)  # Skip the first line of headers
            for line in file:
                columns = line.strip().split(',')
                    
                if 'Data de Atendimento:' in line:
                    for col in columns[1:]:
                        if col:
                            data_atendimento = datetime.datetime.strptime(col, "%d/%m/%Y").strftime('%Y/%m/%d')
                            anomes = datetime.datetime.strptime(col, "%d/%m/%Y").strftime('%Y/%m')

                if line[1].isdigit():
                    if len(columns) == 0:
                        pass
                    elif len(columns) == 11:
                        atendimento_codigo = columns[0]
                        paciente_codigo = columns[3]
                        paciente_nome = columns[4]
                        hora = columns[6]
                        convenio = columns[7]
                        medico_codigo = columns[8]
                        medico_nome = columns[9]
                        centro_de_custos = columns[10]
                    elif len(columns) == 12:
                        atendimento_codigo = columns[0]
                        paciente_codigo = columns[2]
                        paciente_nome = columns[4]
                        hora = columns[6]
                        convenio = columns[7]
                        medico_codigo = columns[8]
                        medico_nome = columns[9]
                        centro_de_custos = columns[10]
                    elif len(columns) == 13:
                        atendimento_codigo = columns[0]
                        paciente_codigo = columns[3]
                        paciente_nome = columns[5]
                        hora = columns[7]
                        convenio = columns[8]
                        medico_codigo = columns[9]
                        medico_nome = columns[10]
                        centro_de_custos = columns[11]
                    elif len(columns) == 14:
                        atendimento_codigo = columns[0]
                        paciente_codigo = columns[2]
                        paciente_nome = columns[5]
                        hora = columns[8]
                        convenio = columns[9]
                        medico_codigo = columns[10]
                        medico_nome = columns[11]
                        centro_de_custos = columns[12]

                    else:
                        print(f'Line not recognized: {len(columns)}, {columns}')
                    try:
                        # Assuming the structure is consistent with the observed pattern
                        processed_data.append({
                            'MES': anomes, 
                            'DIA': f'{data_atendimento} {hora}',
                            'ATENDIMENTO_CODIGO': atendimento_codigo,
                            'PACIENTE_CODIGO': paciente_codigo,
                            'PACIENTE': paciente_nome, 
                            'CONVENIO': convenio,
                            'MEDICO_CODIGO': medico_codigo,
                            'MEDICO': medico_nome, 
                            'CENTRO_CUSTOS': centro_de_custos, 
                        })
                    except Exception as e:
                        print(line)
                        # Handle the case where there are not enough columns in a row
                        print(f"Row with missing data: {line}")
        df = pd.DataFrame(processed_data)
        df.to_csv(f'{file_path}_cleaned.csv', index=False)

        dfs.append(df)

    print('Obrigado Santo Dr. Fausto por limpar esse arquivo CSV do "maravilhoso" MV para mim :) Amém!')

    return dfs
dfs = clean_mv(['MV Todos prestadores.csv'])


In [ ]:
dfs[0]

In [15]:
!pip install docx -q -U
!pip install pip install python-docx -q -U

In [ ]:
import re
from datetime import datetime
import pandas as pd
from docx import Document
import unidecode
import string

formatting_to_status = {
    'None 8C98A7': 'PRIMEIRA',
    'True 8C98A7': 'FALTANTE',
    'None 8FAD70': 'LIVRE',
    'None A9B2BD': 'BLOQUEADO',
    'None 7C78B2': 'A DEFINIR'
}

def sys_clean_text(text):
    """
    Cleans text by removing any leading/trailing white space, converting it to lowercase, removing
    accents, punctuation, and converting to uppercase.
    
    Args:
    text (str): The input text to clean.
    
    Returns:
    str: The cleaned text.
    """
    try:
        # Convert text to string
        try:
            text = str(text)
        except Exception as e:
            print(text, 'is not convertible')
        # Remove accents, punctuation, and convert to uppercase
        text = unidecode.unidecode(text).translate(str.maketrans('', '', string.punctuation)).upper().strip()
        # Replace multiple spaces with a single space
        text = re.sub(r'\s+', ' ', text)
    except Exception as e:
        print(e)
    return text

def extract_run_formatting(run):
    # Determine strike-through status
    strike = 'True' if run.font.strike else 'None'
    # Get the color of the text
    color = run.font.color.rgb if run.font.color else 'None'
    # Combine the formatting into a string
    formatting = f'{strike} {color}'
    # Apply the mapping to convert formatting to status
    status = formatting_to_status.get(formatting, 'UNKNOWN')
    
    return status

# Função para verificar se uma string está no formato de data dd/mm
def parse_date(string):
    try:
        # Tentar converter a string para um objeto datetime
        return datetime.strptime(string, '%d/%m/%Y')
    except ValueError:
        # Retornar None se a string não for uma data válida
        return None

# Função para processar o documento e criar o dataframe
def create_dataframe_from_docx(paragraphs):
    data = []
    current_date = None

    for paragraph in paragraphs:
        text = paragraph.text.strip()
        # Attempt to convert text to date
        date_attempt = parse_date(text)
        if date_attempt:
            current_date = date_attempt  # Update current date if it's a valid date
        else:
            match = re.search(r'(\d{2}:\d{2}) (.+?) 1ª CONSULTA', text)
            if match and current_date:
                time = match.group(1)
                patient_name = match.group(2).strip()
                formatting = [extract_run_formatting(run) for run in paragraph.runs if patient_name in run.text][0]
                # Format the date and time into MES and DIA format
                mes = current_date.strftime('%Y/%m')
                dia = f"{current_date.strftime('%Y/%m/%d')} {time}"
                # Clean the patient name
                paciente_cleaned = sys_clean_text(patient_name)
                data.append([mes, dia, paciente_cleaned, formatting])
    
    # Create dataframe with new column names
    df = pd.DataFrame(data, columns=['MES', 'DIA', 'PACIENTE', 'GERCON'])
    return df

# Carregar o documento docx
doc = Document('GERCON 12 meses.docx')

# Criar o dataframe a partir do documento
df_gercon = create_dataframe_from_docx(doc.paragraphs)
df_gercon.to_csv('df_gercon.csv', index=False)
df_gercon

In [ ]:
df['GERCON'].unique()

In [ ]:
df_joined = pd.merge(df_gercon, df_crv, on='PACIENTE', how='inner')


In [65]:
def standardize_date_adjusted(date_str):
    if pd.isnull(date_str):
        return None
    parts = date_str.split('/')
    # Se a data está incompleta ou em um formato diferente, ajusta para o formato correto
    if len(parts) == 2:  # Se apenas dia e mês são fornecidos, assumimos o ano atual (2023).
        parts.append('2023')
    if len(parts[0]) == 4:  # Se a data está no formato YYYY/MM/DD, reorganize para DD/MM/YYYY
        return f"{parts[2]}/{parts[1]}/{parts[0]}"
    # Adiciona zero ao dia ou mês se necessário
    day = parts[0].zfill(2)
    month = parts[1].zfill(2)
    # Corrige o ano, se estiver em dois dígitos ou incorreto
    year = parts[2] if len(parts[2]) == 4 else f"20{parts[2]}"
    # Corrige anos conhecidos que estão incorretos
    if year in ['2028', '2003', '1980']:
        year = '2023'
    return f"{day}/{month}/{year}"

df_recursos_opticos = pd.read_csv('recursos_opticos.csv')
df_recursos_opticos['VALOR'] = df_recursos_opticos['VALOR'].str.replace(',', '.').astype(float)

# Aplicar a função de padronização ajustada às colunas 'TESTE' e 'PEDIDO'
df_recursos_opticos['TESTE'] = df_recursos_opticos['TESTE'].apply(standardize_date_adjusted)
df_recursos_opticos['PEDIDO'] = df_recursos_opticos['PEDIDO'].apply(standardize_date_adjusted)

# Tentar converter as colunas 'TESTE' e 'PEDIDO' para datetime novamente
df_recursos_opticos['TESTE'] = pd.to_datetime(df_recursos_opticos['TESTE'], format='%d/%m/%Y', errors='coerce')
df_recursos_opticos['PEDIDO'] = pd.to_datetime(df_recursos_opticos['PEDIDO'], format='%d/%m/%Y', errors='coerce')

# Sanitizar os nomes de paciente
df_recursos_opticos['PACIENTE'] = df_recursos_opticos['PACIENTE'].apply(sys_clean_text)


In [66]:
from fuzzywuzzy import process
import pandas as pd

df_mv = pd.read_csv('df_mv.csv')

def correct_name(wrong_names, correct_names, threshold=90):
    name_corrections = {}
    
    for wrong_name in wrong_names:
        match, similarity = process.extractOne(wrong_name, correct_names)
        
        if similarity >= threshold:
            name_corrections[wrong_name] = match
            # print(f'{wrong_name} -> {match} {similarity}%')
        else: 
            print(f'{wrong_name} -> {match}? {similarity}%')
    
    return name_corrections

manual_name_corrections = {
    'ARIEL LACERDA KONSKY': 'ARIEL DE LACERDA KOLESNY',
    'ALESSANDRO PINHEIRO MATTOS': 'ALESSANDRO PINHEIRO DE MAT',
    'CARMEN REGINA RODRIGUES ISRAEL': 'CARMEN REGINA DE OLVEIRA G', 
    'ARISTEU SOARES NETO': 'ARISTEU SOARES CASTRO', 
    'ALICE DA SILVA MENEZES': 'ALICE KREFTA DA SILVA MENEZE', 
    # 'FRANCIELE CRISTINA RODRIGUES DOS SANTOS': 'FRANCIELE CRISTINA RODRIGUE', 
    # adicione todos os outros nomes que precisam ser corrigidos
}
df_recursos_opticos['PACIENTE'] = df_recursos_opticos['PACIENTE'].apply(lambda x: manual_name_corrections.get(x, x))

# Converting names to lowercase for comparison to avoid case sensitivity issues
unique_names_mv = set(df_mv['PACIENTE'].unique())
unique_names_recursos_opticos = set(df_recursos_opticos['PACIENTE'].unique())

mv_only = unique_names_mv - unique_names_recursos_opticos
recursos_opticos_only = unique_names_recursos_opticos - unique_names_mv

# Obtaining the correction dictionary
name_corrections_dict = correct_name(recursos_opticos_only, unique_names_mv)

# Applying the name corrections to df_ro
df_recursos_opticos['PACIENTE'] = df_recursos_opticos['PACIENTE'].apply(lambda x: name_corrections_dict.get(x, x))

# Atualizar os conjuntos de nomes após a aplicação das correções
updated_unique_names_ro = set(df_recursos_opticos['PACIENTE'].unique())

# Encontrar nomes que ainda estão apenas em recursos ópticos e não em mv (ou vice-versa)
remaining_ro_only = updated_unique_names_ro - unique_names_mv
remaining_mv_only = unique_names_mv - updated_unique_names_ro
remaining_ro_only

ARISTEU SOARES NETO -> TAIS PATRICIA SOARES DA SILVA? 86%
SETRO CRV -> PEDRO CARVALHO SIQUEIRA? 70%
FRANCIELE CRISTINA RODRIGUES DOS SANTOS -> DANIEL RODRIGUES FLORES? 86%
ALESSANDRO PINHEIRO MATTOS -> ALESSANDRO PINHEIRO DE MAT? 88%
ARIEL LACERDA KONSKY -> ARIEL DE LACERDA KOLESNY? 82%
SETOR CRV -> PEDRO CARVALHO SIQUEIRA? 60%
CARMEN REGINA RODRIGUES ISRAEL -> CARMEM REGINA RODRIGUEZ IS? 86%
ALICE DA SILVA MENEZES -> LACI DA SILVA? 86%


{'ALESSANDRO PINHEIRO MATTOS',
 'ALICE DA SILVA MENEZES',
 'ARIEL LACERDA KONSKY',
 'ARISTEU SOARES NETO',
 'CARMEN REGINA RODRIGUES ISRAEL',
 'FRANCIELE CRISTINA RODRIGUES DOS SANTOS',
 'SETOR CRV',
 'SETRO CRV'}

In [68]:
df_recursos_opticos.to_csv('df_recursos_opticos_cleaned.csv', index=False)

In [69]:
# Dictionary for supplier's price list with items as keys and prices as values
supplier_prices = {
    "OCULOS ASFERICOS": 645.00,  # Assumed common price for different dioptrias
    "OCULOS PRISMATICOS ARO PQ 10/12": 645.00,
    "OCULOS PRISMATICOS ARO PQ 12/14": 645.00,
    "OCULOS PRISMATICO ARO PQ 4/6": 645.00,
    "OCULOS PRISMATICOS ARO PQ 6/8": 645.00,
    "OCULOS PRISMATICOS ARO PQ 8/10": 645.00,
    "4X12 TC MONOCULAR": 528.00,
    "LUPA DE APOIO 80MM AS 3X LED BAT.": 655.00,
    "LUPA DE MAO 24D 6X 58MM LED": 495.00,
    "LUPA DE MAO 35MM 7X LED": 427.00,
    "LUPA MAX TV": 573.00,
    "LUPA PESO DE PAPEL 2X": 519.00,  # Assuming the closest item "LUPA TIPO PEDRA 62 cm 4x" price
    "LUPA DE APOIO MEDIA 4X 64 MM": 350.00  # Assuming the closest item "LUPA DE APOIO 80MM AS 3X LED BAT." price
}

# Dictionary for items and quantities from invoice 13.778
invoice_13778_items = {
    "OCULOS ASFERICOS": 1,
    "OCULOS PRISMATICOS ARO PQ 10/12": 4,
    "OCULOS PRISMATICOS ARO PQ 12/14": 4,
    "OCULOS PRISMATICO ARO PQ 4/6": 3,
    "OCULOS PRISMATICOS ARO PQ 6/8": 5,
    "OCULOS PRISMATICOS ARO PQ 8/10": 12,
    "4X12 TC MONOCULAR": 12
}

# Dictionary for items and quantities from invoice 13.714
invoice_13714_items = {
    "LUPA DE APOIO 80MM AS 3X LED BAT.": 1,
    "LUPA DE MAO 24D 6X 58MM LED": 10,
    "LUPA DE MAO 35MM 7X LED": 6,
    "LUPA MAX TV": 2,
    "LUPA PESO DE PAPEL 2X": 1,
    "LUPA DE APOIO MEDIA 4X 64 MM": 1
}

# Combine the invoice items and quantities with the supplier prices
# We will create a new dictionary to store items with their common quantities and prices
combined_items = {}

# Combine items from invoice 13778
for item, qty in invoice_13778_items.items():
    if item in supplier_prices:
        combined_items[item] = {
            "quantity": qty,
            "unit_price": supplier_prices[item]
        }

# Combine items from invoice 13714
for item, qty in invoice_13714_items.items():
    if item in supplier_prices:
        # If the item is already in the combined_items, we add the quantities
        if item in combined_items:
            combined_items[item]["quantity"] += qty
        else:
            combined_items[item] = {
                "quantity": qty,
                "unit_price": supplier_prices[item]
            }

combined_items


{'OCULOS ASFERICOS': {'quantity': 1, 'unit_price': 645.0},
 'OCULOS PRISMATICOS ARO PQ 10/12': {'quantity': 4, 'unit_price': 645.0},
 'OCULOS PRISMATICOS ARO PQ 12/14': {'quantity': 4, 'unit_price': 645.0},
 'OCULOS PRISMATICO ARO PQ 4/6': {'quantity': 3, 'unit_price': 645.0},
 'OCULOS PRISMATICOS ARO PQ 6/8': {'quantity': 5, 'unit_price': 645.0},
 'OCULOS PRISMATICOS ARO PQ 8/10': {'quantity': 12, 'unit_price': 645.0},
 '4X12 TC MONOCULAR': {'quantity': 12, 'unit_price': 528.0},
 'LUPA DE APOIO 80MM AS 3X LED BAT.': {'quantity': 1, 'unit_price': 655.0},
 'LUPA DE MAO 24D 6X 58MM LED': {'quantity': 10, 'unit_price': 495.0},
 'LUPA DE MAO 35MM 7X LED': {'quantity': 6, 'unit_price': 427.0},
 'LUPA MAX TV': {'quantity': 2, 'unit_price': 573.0},
 'LUPA PESO DE PAPEL 2X': {'quantity': 1, 'unit_price': 519.0},
 'LUPA DE APOIO MEDIA 4X 64 MM': {'quantity': 1, 'unit_price': 350.0}}

In [ ]:
import fitz  # PyMuPDF
import pandas as pd

# Open the PDF file
folder = r'C:\Users\faust\Downloads'
pdf_path = folder + '\' + r'LISTA DE PREÇOS ATUAL 2023 - BAIXA VISÃO - SOMENTE REVENDA.pdf'
pdf_document = fitz.open(pdf_path)

# Read the text of each page
text_content = ''
for page_number in range(len(pdf_document)):
    page = pdf_document.load_page(page_number)
    text_content += page.get_text()

# Close the PDF after extraction
pdf_document.close()

# Save the text to a .txt file
txt_path = '/mnt/data/extracted_text.txt'
with open(txt_path, 'w') as txt_file:
    txt_file.write(text_content)

# Provide the path to the extracted text file
txt_path
